In [160]:
from ACLScheduleTest.Simulate import Simulator
from ACLScheduleTest.Analysis import Analyst
from ACLScheduleTest.data import RandomDsk
import numpy as np
from time import time

rd = RandomDsk(
    mute=True, node_num=100,
    st_node_num=10,
    min_time=1, time_lv=0, time_rv=0, min_time_step=0,
    avg_mem_peek=1, mem_pv=0, mem_sv=1
)
dsk = rd.build()
sim = Simulator()

In [161]:
num_workers, num_nodes = 10, 100

information = np.ones((num_nodes, num_workers))
def topoOrder(workers, cache, av_mem, factor_node_dict, calculated_node_dict, dsk, global_dict):
    topoQueue = []
    matrix = np.zeros((num_nodes, num_nodes))
    nodeList = list(dsk.keys())
    for idx, node in enumerate(nodeList):
        for n in dsk[node][1:]:
            if n in nodeList:
                id = nodeList.index(n)
                matrix[idx, id] = 1 # idx depends on id
    while len(topoQueue) < len(nodeList):
        for idx, node in enumerate(nodeList):
            if node in topoQueue:
                continue
            if matrix[idx].sum() == 0:
                topoQueue.append(node)
                matrix[:, idx] = 0
    return topoQueue


## Ant as Agent

In [174]:
current_worker_id = 0
iter = 0
from schedule_func_backup import schedule_out_func
def search():
    global current_worker_id, iter
    current_worker_id, iter = 0, 0
    worker_schedules = []
    for idx in range(num_workers):
        worker_schedule = []
        worker_schedules.append(worker_schedule)
    for node_id in range(num_nodes):
        prob = information[node_id] / information[node_id].sum()
        select_worker = np.random.choice(range(num_workers), p=prob)
        worker_schedules[select_worker].append(node_id)
    print(worker_schedules)
    def search_n_func(workers, cache, av_mem, factor_node_dict, calculated_node_dict, dsk, global_dict):
        global current_worker_id, iter
        if len(worker_schedules[current_worker_id])==0:
            return None
        node_id = worker_schedules[current_worker_id][0]
        node = f'node{node_id}'
        ori_id = current_worker_id
        current_worker_id += 1
        if current_worker_id >= num_workers:
            current_worker_id = 0
            iter += 1
            print(f'\niter {iter}')
        #print(current_worker_id)
        nodeList = set(dsk.keys())
        #print(node)

        pre_nodes = [arg for arg in dsk[node][1:] if arg in nodeList]
        for arg in pre_nodes:
            if arg in nodeList and arg not in cache.keys():
                print('None')
                return None
        worker_schedules[ori_id].remove(node_id)
        print(node)
        return node
    result = sim.simulate(search_n_func, schedule_out_func, mem_bound=50, worker_bound=10, rd=rd)
    print(result)

## Train

In [175]:
def train(num_epochs, num_agent):
    for epoch in range(num_epochs):
        for agent_idx in range(num_agent):
            search()
train(1,1)

[[6, 32, 35, 37, 49, 50, 57, 63, 66, 72, 76, 89, 90, 91, 98], [28, 34, 74, 87, 94], [14, 15, 16, 22, 24, 26, 40, 43, 48, 55, 83, 85, 93], [25, 33, 38, 69, 77, 82, 99], [41, 42, 45, 52, 53, 61, 67, 71, 97], [0, 1, 51, 54, 56, 80, 81], [4, 8, 21, 31, 60, 64, 75, 78], [2, 5, 7, 9, 10, 18, 19, 20, 23, 27, 30, 39, 47, 70, 73, 95], [3, 17, 36, 44, 46, 58, 59, 84, 86, 88, 92], [11, 12, 13, 29, 62, 65, 68, 79, 96]]
node6
None
None
None
None
node0
node4
node2
node3

iter 1
None
None
None
node14
None
None
node1
node8
node5
None

iter 2
None
None
None
None
None
None
None
None
node7
node17

iter 3
None
None
None
None
None
None
None
None
node9
None

iter 4
node11
None
None
node15
None
None
None
node21
node10
None

iter 5
None
None
None
node16
None
None
None
node31
None
None

iter 6
node12
node32
None
node22
node25
None
None
None
node18
None

iter 7
node13
None
None
node24
node33
None
None
None
node19
node36

iter 8
None
None
None
None
node38
None
node51
None
node20
None

iter 9
node29
node35
None
N

## schedule_in_func 

## schedule_out_func